In [ ]:
from gli import GraphStore, GLI, create_random_graph

In [ ]:
# Test with the new GraphStore name
store = GraphStore()

# Create a simple graph
graph = create_random_graph(5, 0.4)
store.update_graph(graph, "create_random_graph")

# Commit this state
store.commit("Initial random graph")

# Add a node
graph = store.get_current_graph()
graph = graph.add_node("new_node", color="red")
store.update_graph(graph, "add_node")

# Commit this state
store.commit("Added a new node")

# Show history
print("History:")
for entry in store.get_history():
    print(f"  {entry['hash']}: {entry['state'].operation}")

print(f"\nCurrent graph has {len(store.get_current_graph().nodes)} nodes")
print(f"Content pool stats: {store.get_storage_stats()}")

History:
  initial: initialize
  d9b48f215b0cbf3a: create_random_graph
  fc2688e2d94a45e2: add_node

Current graph has 6 nodes


In [ ]:
store.commits

{'d9b48f215b0cbf3a': {'message': 'Initial random graph',
  'timestamp': 1750862679.685036},
 'fc2688e2d94a45e2': {'message': 'Added a new node',
  'timestamp': 1750862679.6853209}}

In [ ]:
store.get_current_graph().nodes

OrderedDict([('node_0', Node(id='node_0', attributes={})),
             ('node_1', Node(id='node_1', attributes={})),
             ('node_2', Node(id='node_2', attributes={})),
             ('node_3', Node(id='node_3', attributes={})),
             ('node_4', Node(id='node_4', attributes={})),
             ('new_node', Node(id='new_node', attributes={'color': 'red'}))])

In [ ]:
store.states

{'initial': GraphState(hash='initial', parent_hash=None, operation='initialize', timestamp=1750862679.684452, nodes={}, edges={}, graph_attributes={}, delta=None),
 'd9b48f215b0cbf3a': GraphState(hash='d9b48f215b0cbf3a', parent_hash='initial', operation='create_random_graph', timestamp=1750862679.6849952, nodes={'node_0': '1fcc9959e57c7d3e', 'node_1': '2ac655daf3db1da9', 'node_2': '14d87a1119463da8', 'node_3': '46567b629541d3d7', 'node_4': '3bb76ac318812752'}, edges={'node_0->node_2': '7fcf3a8fbefcaf5d', 'node_0->node_3': 'e0773d9e6370398d', 'node_1->node_2': '4f6188463821ac2c', 'node_3->node_4': 'b1b2935fed49cfe3'}, graph_attributes={}, delta=None),
 'fc2688e2d94a45e2': GraphState(hash='fc2688e2d94a45e2', parent_hash='d9b48f215b0cbf3a', operation='add_node', timestamp=1750862679.685288, nodes={'node_0': '1fcc9959e57c7d3e', 'node_1': '2ac655daf3db1da9', 'node_2': '14d87a1119463da8', 'node_3': '46567b629541d3d7', 'node_4': '3bb76ac318812752', 'new_node': '658141e053860dd6'}, edges={'nod

In [ ]:
# Test content pool isolation
print("=== CONTENT POOL ISOLATION TEST ===")

# Create two separate GraphStore instances
store1 = GraphStore()
store2 = GraphStore()

# Each should have its own content pool
print(f"Store1 content pool nodes: {len(store1.content_pool.nodes)}")
print(f"Store2 content pool nodes: {len(store2.content_pool.nodes)}")

# Add different graphs to each
graph1 = store1.current_graph.add_node("store1_node").snapshot()
graph2 = store2.current_graph.add_node("store2_node").snapshot()

store1.update_graph(graph1, "add_to_store1")
store2.update_graph(graph2, "add_to_store2")

print(f"After updates:")
print(f"Store1 content pool nodes: {len(store1.content_pool.nodes)}")
print(f"Store2 content pool nodes: {len(store2.content_pool.nodes)}")
print(f"Store1 graph nodes: {list(store1.get_current_graph().nodes.keys())}")
print(f"Store2 graph nodes: {list(store2.get_current_graph().nodes.keys())}")

Testing create_random_graph:
Nodes before snapshot: 5
Has pending delta: False
Nodes after snapshot: 5
Node list: ['node_0', 'node_1', 'node_2', 'node_3', 'node_4']


In [ ]:
# Test fresh GraphStore instance with proper snapshots
print("Creating fresh GraphStore instance:")
store3 = GraphStore()

# Create graph and force snapshot
graph = create_random_graph(3, 0.5)
print(f"Random graph created with {len(graph.nodes)} visible nodes")
print(f"Pending delta: {graph._pending_delta is not None}")

# Force snapshot before updating GraphStore
snapshot_graph = graph.snapshot()
print(f"Snapshot has {len(snapshot_graph.nodes)} nodes")

store3.update_graph(snapshot_graph, "create_random_graph")
store3.commit("Initial random graph")

print(f"GraphStore current graph has {len(store3.get_current_graph().nodes)} nodes")
print(f"Node IDs: {list(store3.get_current_graph().nodes.keys())}")

Creating fresh GLI instance:
Random graph created with 3 visible nodes
Pending delta: False
Snapshot has 3 nodes
GLI current graph has 3 nodes
Node IDs: ['node_0', 'node_1', 'node_2']


In [ ]:
# Test the issue with node addition
print("Testing node addition:")
current = store3.get_current_graph()
print(f"Before adding: {len(current.nodes)} nodes")

# Add a node
modified = current.add_node("test_node", color="blue")
print(f"After adding (before snapshot): {len(modified.nodes)} visible nodes")
print(f"Has pending delta: {modified._pending_delta is not None}")

# Apply snapshot
final = modified.snapshot()
print(f"After snapshot: {len(final.nodes)} nodes")

# Update GraphStore
store3.update_graph(final, "add_test_node")
print(f"GraphStore after update: {len(store3.get_current_graph().nodes)} nodes")
print(f"All nodes: {list(store3.get_current_graph().nodes.keys())}")

Testing node addition:
Before adding: 3 nodes
After adding (before snapshot): 3 visible nodes
Has pending delta: True
After snapshot: 4 nodes
GLI after update: 4 nodes
All nodes: ['node_0', 'node_1', 'node_2', 'test_node']


In [ ]:
# Test backward compatibility
print("=== BACKWARD COMPATIBILITY TEST ===")

# GLI should still work as an alias
old_gli = GLI()
print(f"GLI alias works: {type(old_gli).__name__}")

# Should have same functionality
graph = create_random_graph(2, 0.5)
old_gli.update_graph(graph, "test_alias")
print(f"GLI alias graph nodes: {len(old_gli.get_current_graph().nodes)}")

=== COMPLETE DEBUGGING TEST ===

1. Creating fresh GLI...
Initial state: initial
Initial nodes: 0

2. Creating random graph manually...
Empty graph nodes: 0
After adding manual_0: visible=0, pending=True
After adding manual_1: visible=0, pending=True
After adding manual_2: visible=0, pending=True
Manual snapshot nodes: 3
Manual node list: ['manual_0', 'manual_1', 'manual_2']

3. Updating GLI with manual graph...
New hash: a8576719676a96fc
GLI nodes after update: 3
GLI node list: ['manual_0', 'manual_1', 'manual_2']

4. Checking states...
State initial: root=True, nodes=0, delta=False
State a8576719676a96fc: root=False, nodes=3, delta=False
  Node hashes: ['manual_0', 'manual_1', 'manual_2']


In [ ]:
# Test the final working version
print("=== FINAL WORKING TEST ===")

# Fresh GraphStore instance
store_final = GraphStore()

# Test 1: Simple graph
simple = store_final.current_graph.add_node("test1").add_node("test2").snapshot()
hash1 = store_final.update_graph(simple, "add_simple")
print(f"Simple graph: {len(store_final.get_current_graph().nodes)} nodes")
print(f"Nodes: {list(store_final.get_current_graph().nodes.keys())}")

# Test 2: Random graph
random_graph = create_random_graph(3, 0.5)
hash2 = store_final.update_graph(random_graph, "add_random")
print(f"Random graph: {len(store_final.get_current_graph().nodes)} nodes")
print(f"Nodes: {list(store_final.get_current_graph().nodes.keys())}")

# Test 3: Add more nodes
current = store_final.get_current_graph()
modified = current.add_node("extra1").add_node("extra2", color="blue")
hash3 = store_final.update_graph(modified, "add_extra")
print(f"With extras: {len(store_final.get_current_graph().nodes)} nodes")
print(f"All nodes: {list(store_final.get_current_graph().nodes.keys())}")

# Check all states work
print(f"\nAll states: {list(store_final.states.keys())}")
for state_hash in store_final.states.keys():
    reconstructed = store_final._reconstruct_graph_from_state(state_hash)
    print(f"State {state_hash}: {len(reconstructed.nodes)} nodes")

# Show storage stats
print(f"\nStorage stats: {store_final.get_storage_stats()}")

=== CONTENT POOL DEBUG ===
Content pool has 10 nodes
Content pool has 4 edges

1. Testing simple graph creation...
Simple graph nodes: 1
After update, GLI hash: 9162ea9f3196876d
State is root: False
State has 1 node hashes
  Node test -> hash 5a50f1eeafb8f548
    Pool lookup: Node(id='test', attributes={'value': 123})

2. Manual reconstruction test...
Reconstructed graph has 1 nodes


=== TESTING THE FIX ===
Simple graph: 2 nodes
Nodes: ['test1', 'test2']
Random graph: 3 nodes
Nodes: ['node_0', 'node_1', 'node_2']
With extras: 5 nodes
All nodes: ['node_0', 'node_1', 'node_2', 'extra1', 'extra2']

All states: ['initial', '15a7b01ddea833cf', '012faf713354f3f1', '54c2265ec7e3c802']
State initial: 0 nodes
State 15a7b01ddea833cf: 2 nodes
State 012faf713354f3f1: 5 nodes
State 54c2265ec7e3c802: 5 nodes
